# Week 14: Poly Cube Mapping Attempt
A naive test on poly-cube mapping

## Table of Contents
- [0 - Packages and Resources](#0)
- [1 - Implementation](#1)
    - [*1.0 - The Example](#1-0)
    - [1.1 - Area Calculation](#1-1)
    - [1.2 - BFS](#1-2)
    - [1.3 - Fit a Plane](#1-3)
    - [1.4 - Intersection of Plane and Mesh](#1-4)
- [2 - Sample Outcome](#2)
    - [2.1 - One Round](#2-1)
    - [2.2 - Convergence](#2-2)
- [3 - Reference](#3)

<a name='0'></a>
## 0. Packages and Resources

In [31]:
# Packages
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Self-defined functions
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from util.util import distance_euclidean
from util.mesh.triangle.R3 import calculate_single_dihedral_angle, calculate_all_dihedral_angles
from util.mesh.triangle.common import retrieve_all_edges

# Visualization
import pyvista as pv
from pyvista import examples

<a name='1'></a>
## 1. Implementation

<a name='1-0'></a>
### *1.0 The Example

Here I use the sphere as an example.

In [22]:
sphere = pv.Sphere(radius=5, theta_resolution=10, phi_resolution=10)

faces = sphere.faces.reshape((-1,4))[:, 1:4]
vertices = sphere.points
angles = calculate_all_dihedral_angles(faces, vertices)
edges = retrieve_all_edges(faces)

pl = pv.Plotter()
pl.add_mesh(sphere, show_edges=True, color="white")
pl.add_points(sphere.points, color="red", point_size=5)
# For monitor purpose
# pl.add_points(sphere.points[55], color="blue", point_size=20)
# pl.add_points(sphere.points[64], color="blue", point_size=20)
pl.show()

Widget(value="<iframe src='http://localhost:55106/index.html?ui=P_0x27f5baa4bb0_2&reconnect=auto' style='width…

In [36]:
class chen_2023:
    def __init__(self, vertices, faces, lambda1=1, lambda2=1, max_iter = 30):
        self.max_iter = max_iter
        self.vertex_num = vertices.shape[0]
        self.vertices = vertices
        self.faces = faces
        self.lambda1=lambda1
        self.lambda2=lambda2
        
        self.solution = self.vertices.copy()
        self.edges = retrieve_all_edges(faces)
    
    def activation(self, angle):
        if angle < (np.pi/2):
            return np.power(np.cos(angle), 2) / angle
        else:
            return np.power(np.cos(angle), 2)
        
    def loss_classification(self, x):
        X = x.reshape((self.vertex_num , 3))
        
        EB = 0
        for edge in self.edges:
            A, B = X[edge[0]], X[edge[1]]
            EB += np.sum(np.absolute(B - A)) - distance_euclidean(A, B)
            
        EA = 0
        dihedral_angles = calculate_all_dihedral_angles(faces, X)
        for angle_value in dihedral_angles.values():
            EA += self.activation(angle_value)
            
        return self.lambda1*EB + self.lambda2*EA
    
    def optimize(self):
        x0 = np.ravel(self.solution)
        self.res = minimize(self.loss_classification, x0, options = {'maxiter': self.max_iter})
        self.solution = self.res.x.reshape((self.vertex_num, 3))
    
    def optimize_one_round(self):
        x0 = np.ravel(self.solution)
        self.res = minimize(self.loss_classification, x0, options = {'maxiter': 1})
        self.solution = self.res.x.reshape((self.vertex_num, 3))
        
#     def visualize_initial(self, show_boundary = False):
#         plt.plot(self.vertices[:,0], self.vertices[:,1], 'o', color='green')
#         plt.triplot(self.vertices[:,0], self.vertices[:,1], self.faces, label='Original Mesh', color='orange')
#         if show_boundary:
#             for edge in self.boundary_edges:
#                 plt.plot(self.vertices[np.array(edge), 0], self.vertices[np.array(edge), 1], 'y-')
#         plt.axis('equal')
#         plt.legend()
        
#     def visualize_solution(self, show_boundary = False):
#         plt.plot(self.solution[:,0], self.solution[:,1], 'o', color='blue')
#         plt.triplot(self.solution[:,0], self.solution[:,1], self.faces, label='Transformed Mesh', color='blue')
#         if show_boundary:
#             for edge in self.boundary_edges:
#                 plt.plot(self.solution[np.array(edge), 0], self.solution[np.array(edge), 1], 'r-')
#         plt.axis('equal')
#         plt.legend()
        
#     def visualize(self):
#         self.visualize_initial()
#         self.visualize_solution()

In [43]:
test = chen_2023(vertices.copy(), faces.copy(), max_iter = 30)

In [44]:
test.optimize()

In [45]:
mesh_update = pv.PolyData(test.solution, sphere.faces)

In [50]:
p = pv.Plotter()
p.add_mesh(mesh_update, show_edges=True, color="")
p.add_mesh(sphere, show_edges=True, color="white")
p.add_points(mesh_update.points, color="red", point_size=5)
p.show()

Widget(value="<iframe src='http://localhost:55106/index.html?ui=P_0x27f67563b20_9&reconnect=auto' style='width…